In [1]:
from app.response import Response
from app.models import User, Conf, Catalog, Image, Tag
from app import db

### init dataset

In [2]:
db.drop_all()
db.create_all()

In [2]:
u = User(username='admit',email="admit@test.com")

In [3]:
db.session.add(u)
db.session.commit()

In [4]:
res = Response(u)

In [5]:
res.init_catalog()

In [6]:
res.init_images()

In [7]:
Catalog.query.all()

[<CAT 1 path: data>]

In [8]:
Image.query.all()

[<Image 1 84651002819.png @ Cat 1, Tag {}>,
 <Image 2 84651013968.png @ Cat 1, Tag {}>,
 <Image 3 84651018070.png @ Cat 1, Tag {}>,
 <Image 4 84651019902.png @ Cat 1, Tag {}>,
 <Image 5 84651020475.png @ Cat 1, Tag {}>,
 <Image 6 84651020533.png @ Cat 1, Tag {}>,
 <Image 7 84651021925.png @ Cat 1, Tag {}>,
 <Image 8 84651022038.png @ Cat 1, Tag {}>,
 <Image 9 84651025036.png @ Cat 1, Tag {}>,
 <Image 10 84651025183.png @ Cat 1, Tag {}>]

In [8]:
cat = Catalog.query.filter_by(name='data').first()

In [9]:
cat.images.all()

[<Image 1 84651002819.png @ Cat 1, Tag {}>,
 <Image 2 84651013968.png @ Cat 1, Tag {}>,
 <Image 3 84651018070.png @ Cat 1, Tag {}>,
 <Image 4 84651019902.png @ Cat 1, Tag {}>,
 <Image 5 84651020475.png @ Cat 1, Tag {}>,
 <Image 6 84651020533.png @ Cat 1, Tag {}>,
 <Image 7 84651021925.png @ Cat 1, Tag {}>,
 <Image 8 84651022038.png @ Cat 1, Tag {}>,
 <Image 9 84651025036.png @ Cat 1, Tag {}>,
 <Image 10 84651025183.png @ Cat 1, Tag {}>]

In [2]:
u = User(username='admit',email="admit@test.com")

In [11]:
db.session.add(u)
db.session.commit()

In [12]:
t1 = Tag(name='A', user=u)
t2 = Tag(name='B', user=u)
t3 = Tag(name='C', user=u)
t4 = Tag(name='N', user=u)

In [13]:
User.query.all()

[<User admit>]

### control

#### get configure

In [14]:
def add_conf(u):
    query = Conf.query.filter_by(user_id=u.id)
    assert query.count() < 2, \
            "There should be only one setting item per user!"
    data = {
        "grade_a": "1",
        "grade_b": "2",
        "grade_c": "3",
        "nonlens": "0",
        "next_im": "j",
        "prev_im": "k",
    }
    if query.count() == 0:
        data["user"] = u
        conf = Conf(**data)
        db.session.add(conf)
    else:
        query.update(data)
    db.session.commit()


In [15]:
add_conf(u)

In [8]:
res.get_current_query(), res.get_current_image()

SELECT image.id AS image_id, image.name AS image_name, image.catalog_id AS image_catalog_id 
FROM image


(None, <Image 1 84651002819.png @ Cat 1, Tag {}>)

In [17]:
res.get_conf(u.id)

In [18]:
res.conf

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7f2ecdd747b8>,
 'grade_c': '3',
 'grade_a': '1',
 'prev_im': 'k',
 'nonlens': '0',
 'grade_b': '2',
 'id': 1,
 'user_id': 1,
 'next_im': 'j'}

### test add label

#### add label on unlabelled image

In [19]:
res.im_id = 1

In [20]:
res.im_query.all()

[<Image 1 84651002819.png @ Cat 1, Tag {}>,
 <Image 2 84651013968.png @ Cat 1, Tag {}>,
 <Image 3 84651018070.png @ Cat 1, Tag {}>,
 <Image 4 84651019902.png @ Cat 1, Tag {}>,
 <Image 5 84651020475.png @ Cat 1, Tag {}>,
 <Image 6 84651020533.png @ Cat 1, Tag {}>,
 <Image 7 84651021925.png @ Cat 1, Tag {}>,
 <Image 8 84651022038.png @ Cat 1, Tag {}>,
 <Image 9 84651025036.png @ Cat 1, Tag {}>,
 <Image 10 84651025183.png @ Cat 1, Tag {}>]

In [21]:
res.keys['1']()
res.keys['2']()
res.keys['3']()
res.keys['0']()

In [22]:
res.im_query.all()

[<Image 1 84651002819.png @ Cat 1, Tag {<User admit>: <Tag A by <User admit>>}>,
 <Image 2 84651013968.png @ Cat 1, Tag {<User admit>: <Tag B by <User admit>>}>,
 <Image 3 84651018070.png @ Cat 1, Tag {<User admit>: <Tag C by <User admit>>}>,
 <Image 4 84651019902.png @ Cat 1, Tag {<User admit>: <Tag N by <User admit>>}>,
 <Image 5 84651020475.png @ Cat 1, Tag {}>,
 <Image 6 84651020533.png @ Cat 1, Tag {}>,
 <Image 7 84651021925.png @ Cat 1, Tag {}>,
 <Image 8 84651022038.png @ Cat 1, Tag {}>,
 <Image 9 84651025036.png @ Cat 1, Tag {}>,
 <Image 10 84651025183.png @ Cat 1, Tag {}>]

#### previous and next method

In [23]:
res.im_id = 2

In [24]:
res.keys['k']()
print(res.get_current_image())

<Image 1 84651002819.png @ Cat 1, Tag {<User admit>: <Tag A by <User admit>>}>


In [25]:
res.keys['j'](),res.keys['j'](),res.keys['j']()
print(res.get_current_image())

<Image 4 84651019902.png @ Cat 1, Tag {<User admit>: <Tag N by <User admit>>}>


In [26]:
res.keys['j']()
print(res.get_current_image())

<Image 5 84651020475.png @ Cat 1, Tag {}>


In [27]:
res.keys['j']()
print(res.get_current_image())

<Image 6 84651020533.png @ Cat 1, Tag {}>


In [28]:
res.im_query.all()

[<Image 1 84651002819.png @ Cat 1, Tag {<User admit>: <Tag A by <User admit>>}>,
 <Image 2 84651013968.png @ Cat 1, Tag {<User admit>: <Tag B by <User admit>>}>,
 <Image 3 84651018070.png @ Cat 1, Tag {<User admit>: <Tag C by <User admit>>}>,
 <Image 4 84651019902.png @ Cat 1, Tag {<User admit>: <Tag N by <User admit>>}>,
 <Image 5 84651020475.png @ Cat 1, Tag {<User admit>: <Tag N by <User admit>>}>,
 <Image 6 84651020533.png @ Cat 1, Tag {}>,
 <Image 7 84651021925.png @ Cat 1, Tag {}>,
 <Image 8 84651022038.png @ Cat 1, Tag {}>,
 <Image 9 84651025036.png @ Cat 1, Tag {}>,
 <Image 10 84651025183.png @ Cat 1, Tag {}>]

#### change label

In [29]:
res.im_id = 2

In [30]:
res.keys['0']()
res.keys['1']()

In [31]:
res.im_query.all()

[<Image 1 84651002819.png @ Cat 1, Tag {<User admit>: <Tag A by <User admit>>}>,
 <Image 2 84651013968.png @ Cat 1, Tag {<User admit>: <Tag N by <User admit>>}>,
 <Image 3 84651018070.png @ Cat 1, Tag {<User admit>: <Tag A by <User admit>>}>,
 <Image 4 84651019902.png @ Cat 1, Tag {<User admit>: <Tag N by <User admit>>}>,
 <Image 5 84651020475.png @ Cat 1, Tag {<User admit>: <Tag N by <User admit>>}>,
 <Image 6 84651020533.png @ Cat 1, Tag {}>,
 <Image 7 84651021925.png @ Cat 1, Tag {}>,
 <Image 8 84651022038.png @ Cat 1, Tag {}>,
 <Image 9 84651025036.png @ Cat 1, Tag {}>,
 <Image 10 84651025183.png @ Cat 1, Tag {}>]

#### check filter

In [32]:
res.filter_untag()
res.im_query.all()

[<Image 6 84651020533.png @ Cat 1, Tag {}>,
 <Image 7 84651021925.png @ Cat 1, Tag {}>,
 <Image 8 84651022038.png @ Cat 1, Tag {}>,
 <Image 9 84651025036.png @ Cat 1, Tag {}>,
 <Image 10 84651025183.png @ Cat 1, Tag {}>]

In [33]:
res.filter_tagA()
res.im_query.all()

[<Image 1 84651002819.png @ Cat 1, Tag {<User admit>: <Tag A by <User admit>>}>,
 <Image 3 84651018070.png @ Cat 1, Tag {<User admit>: <Tag A by <User admit>>}>]

In [34]:
res.filter_tagB()
res.im_query.all()

[]

In [35]:
res.filter_tagN()
res.im_query.all()

[<Image 2 84651013968.png @ Cat 1, Tag {<User admit>: <Tag N by <User admit>>}>,
 <Image 4 84651019902.png @ Cat 1, Tag {<User admit>: <Tag N by <User admit>>}>,
 <Image 5 84651020475.png @ Cat 1, Tag {<User admit>: <Tag N by <User admit>>}>]

#### for test

In [38]:
for i in enumerate([1,2,3]):
    print(i)

(0, 1)
(1, 2)
(2, 3)


In [1]:
from app.response import Response
from app.models import User, Conf, Catalog, Image, Tag
from app import db

In [2]:
u = User.query.filter_by(username="a7").first()

In [3]:
res = Response(u)

In [4]:
res.get_conf()

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f28efd0f278>, 'id': 17, 'user_id': 21, 'next_im': 'j', 'grade_c': '3', 'grade_a': '1', 'prev_im': 'k', 'nonlens': '0', 'grade_b': '2'}
running _conf_method, []
running _conf_method, [<Tag A by <User a7>>, <Tag B by <User a7>>, <Tag C by <User a7>>, <Tag N by <User a7>>]


In [5]:
res.conf['grade_a']

'1'

In [6]:
Conf.query.filter_by(user_id=u.id).all()

[<key map {'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f28efc9bc50>, 'id': 17, 'user_id': 21, 'next_im': 'j', 'grade_c': '3', 'grade_a': '1', 'prev_im': 'k', 'nonlens': '0', 'grade_b': '2'}>]

In [7]:
dict(Conf.query.filter_by(user_id=res.u.id).first().__dict__)

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7f28efc9bc50>,
 'id': 17,
 'user_id': 21,
 'next_im': 'j',
 'grade_c': '3',
 'grade_a': '1',
 'prev_im': 'k',
 'nonlens': '0',
 'grade_b': '2'}

In [18]:
res.conf

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7f31ae4a0978>,
 'id': 14,
 'user_id': 18,
 'next_im': 'j',
 'grade_c': '3',
 'grade_a': '1',
 'prev_im': 'k',
 'nonlens': '0',
 'grade_b': '2'}

In [26]:
res.im_query.order_by(Image.id.desc()).filter(Image.id > 5).all()

[<Image 6 84651020533.png @ Cat 1, Tag {}>,
 <Image 7 84651021925.png @ Cat 1, Tag {}>,
 <Image 8 84651022038.png @ Cat 1, Tag {}>,
 <Image 9 84651025036.png @ Cat 1, Tag {}>,
 <Image 10 84651025183.png @ Cat 1, Tag {}>]

In [6]:
res.im_id = 4

In [24]:
Image.query.order_by(Image.id.desc()).filter(Image.id > 5).all()

[<Image 10 84651025183.png @ Cat 1, Tag {}>,
 <Image 9 84651025036.png @ Cat 1, Tag {}>,
 <Image 8 84651022038.png @ Cat 1, Tag {}>,
 <Image 7 84651021925.png @ Cat 1, Tag {}>,
 <Image 6 84651020533.png @ Cat 1, Tag {}>]